In [1]:
!nvidia-smi

Wed Mar  5 12:12:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             11W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
!pip install transformers datasets nltk

In [1]:
import os
import sys
import transformers
import tensorflow as tf
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

import wandb
wandb.init(mode="disabled")
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [2]:
model_checkpoint = 'snehalyelmati/mt5-hindi-to-english'

In [4]:
#import dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yash9439/eng-hinglish-machine-translation")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/eng-hinglish-machine-translation


In [3]:
def read_file(file_path):
    with open(file_path, "r",encoding='utf-8') as f:
        lines = f.read()
    return lines

test_lines = read_file('/kaggle/input/eng-hinglish-machine-translation/Eng - Hinglish/test.txt')
train_lines = read_file('/kaggle/input/eng-hinglish-machine-translation/Eng - Hinglish/train.txt')
dev_lines = read_file('/kaggle/input/eng-hinglish-machine-translation/Eng - Hinglish/dev.txt')

test_array = test_lines.split("\n")[:-1]
train_array = train_lines.split("\n")[:-1]
dev_array = dev_lines.split("\n")[:-1]

test_split = [line.split("\t") for line in test_array]
train_split = [line.split("\t") for line in train_array]
dev_split = [line.split("\t") for line in dev_array]

df = {}

df['test'] = pd.DataFrame(test_split, columns = ["output", "input"])
df['train'] = pd.DataFrame(train_split, columns = ["output", "input"])
df['dev'] = pd.DataFrame(dev_split, columns = ["output", "input"])

In [4]:
from datasets import Dataset, load_dataset, DatasetDict

ds = DatasetDict()
ds['test'] = Dataset.from_pandas(df['test'])
ds['train'] = Dataset.from_pandas(df['train'])
ds['dev'] = Dataset.from_pandas(df['dev'])


In [7]:
ds

DatasetDict({
    test: Dataset({
        features: ['output', 'input'],
        num_rows: 942
    })
    train: Dataset({
        features: ['output', 'input'],
        num_rows: 7040
    })
    dev: Dataset({
        features: ['output', 'input'],
        num_rows: 1020
    })
})

In [5]:
# TOKENIZER

tokenizer = AutoTokenizer.from_pretrained("snehalyelmati/mt5-hindi-to-english")

tokenizer.model_max_length = 32

def tokenise_df(df_obj):
    return tokenizer.prepare_seq2seq_batch(src_texts = df_obj["input"], tgt_texts = df_obj["output"])

tokenized_datasets = ds.map(tokenise_df, batched = True)

tokenized_datasets

tokenizer_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

Map:   0%|          | 0/942 [00:00<?, ? examples/s]

Map:   0%|          | 0/7040 [00:00<?, ? examples/s]

Map:   0%|          | 0/1020 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['output', 'input', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 942
    })
    train: Dataset({
        features: ['output', 'input', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7040
    })
    dev: Dataset({
        features: ['output', 'input', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1020
    })
})

In [6]:
# SETTING UP MODEL
model = AutoModelForSeq2SeqLM.from_pretrained("snehalyelmati/mt5-hindi-to-english")
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model=model)

config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

In [7]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

def compute_metrics(pred):
    references = pred.label_ids
    generated_texts = pred.predictions
    
    bleu_scores = []
    for reference, generated_text in zip(references, generated_texts):
        reference_text = train_dataset[reference]['text']
        bleu_score = sentence_bleu([reference_text], generated_text)
        bleu_scores.append(bleu_score)

    return {
        'bleu': sum(bleu_scores) / len(bleu_scores)
    }

In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [9]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    eval_strategy="epoch",  # Evaluates at the end of every epoch
    logging_strategy="epoch",  # Logs metrics at the end of every epoch
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    fp16=True,
    gradient_accumulation_steps=8,
    learning_rate=0.001,
    # load_best_model_at_end=True,
    push_to_hub=False,
    save_total_limit=2,
    save_strategy="epoch",
    logging_dir="/kaggle/working/logs",
    resume_from_checkpoint=True,
    
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


In [25]:
# !df -h  # Check disk space usage
# !rm -rf /kaggle/working/results/checkpoint-*  # Delete old checkpoints
# !rm -rf /kaggle/working/logs  # Clear logs if not needed
!rm -rf /kaggle/working/state.db

In [34]:
trainer.train()

Step,Training Loss
55,2.350900
110,1.351100
165,1.154400
220,1.039400
275,0.957700
330,0.888500
385,0.829800
440,0.780800
495,0.736700
550,0.697000


TrainOutput(global_step=1100, training_loss=0.8229520988464355, metrics={'train_runtime': 2478.9449, 'train_samples_per_second': 56.798, 'train_steps_per_second': 0.444, 'total_flos': 4652997279744000.0, 'train_loss': 0.8229520988464355, 'epoch': 20.0})

In [ ]:
import torch
torch.cuda.empty_cache()

training_args.per_device_eval_batch_size = 8
training_args.eval_accumulation_steps = 8  

results = trainer.evaluate()

In [ ]:
results

In [16]:
## PREDICTION
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint_path = "/kaggle/working/results/checkpoint-1100"  # Update with your path

# Load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

# Move model to GPU if available
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [40]:
text = "मैं जिज्ञासा से इसे बनाने के लिए कहूंगा"  # Replace with actual input

# Tokenize the input
inputs = tokenizer(text, return_tensors="pt").to(device)


In [46]:
# Generate response
with torch.no_grad():
    output_ids = model.generate(**inputs, max_length=64)  # Adjust `max_length` as needed

# Decode the generated response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated Response:", response)


Generated Response: I would say it would be a good message to make it


In [26]:
## DOWNLOADING THE CHECKPOINT FOLDER
!zip -r file_logs.zip /kaggle/working/logs

  adding: kaggle/working/logs/ (stored 0%)
  adding: kaggle/working/logs/events.out.tfevents.1741192274.0cde151efdd7.1428.3 (deflated 61%)
  adding: kaggle/working/logs/events.out.tfevents.1741192293.0cde151efdd7.1428.4 (deflated 62%)
  adding: kaggle/working/logs/events.out.tfevents.1741192228.0cde151efdd7.1428.2 (deflated 61%)


In [27]:
from IPython.display import FileLink
FileLink(r'file_logs.zip')

/kaggle/working/file_logs.zip